In [0]:
# Enable outputting results interactively. 
# All the results from code in a given cell will be displayed instead of only the last one.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
import pandas as pd
import re
from datetime import datetime

from bs4 import BeautifulSoup, SoupStrainer
import requests
import pickle

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
url = "http://insideairbnb.com/get-the-data.html"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))

In [0]:
def gather_links(city,cat):

    len(links)
    links_df = pd.DataFrame(links, columns = ['link'])
    links_df2 = links_df[links_df['link'].str.contains(city)==True]
    links_df3 = links_df2[links_df2['link'].str.contains(cat)==True]
    return links_df3

In [0]:
def readin_gz(dt,cat):
    url = 'http://data.insideairbnb.com/united-states/ca/san-francisco/{}/data/{}.csv.gz'.format(str(dt),str(cat))

    listings = pd.read_csv(url,compression='gzip',header=0, sep=',', quotechar='"',dtype= {'zipcode':object})
    return listings
    

In [0]:
links_sf_listings = gather_links('san-francisco','listings.csv.gz')
links_sf_listings['dt'] = [x.split('/')[6] for x in links_sf_listings['link']]
links_sf_listings

idx = links_sf_listings['dt'].unique()
len(idx),idx

,link,dt
7529,http://data.insideairbnb.com/united-states/ca/...,2019-05-03
7536,http://data.insideairbnb.com/united-states/ca/...,2019-04-03
7543,http://data.insideairbnb.com/united-states/ca/...,2019-03-06
7550,http://data.insideairbnb.com/united-states/ca/...,2019-02-01
7557,http://data.insideairbnb.com/united-states/ca/...,2019-01-09
7564,http://data.insideairbnb.com/united-states/ca/...,2018-12-06
7571,http://data.insideairbnb.com/united-states/ca/...,2018-11-03
7578,http://data.insideairbnb.com/united-states/ca/...,2018-10-03
7585,http://data.insideairbnb.com/united-states/ca/...,2018-09-08
7592,http://data.insideairbnb.com/united-states/ca/...,2018-08-06


(45, array(['2019-05-03', '2019-04-03', '2019-03-06', '2019-02-01',
        '2019-01-09', '2018-12-06', '2018-11-03', '2018-10-03',
        '2018-09-08', '2018-08-06', '2018-07-05', '2018-05-09',
        '2018-04-06', '2018-03-04', '2018-02-02', '2018-01-17',
        '2018-01-10', '2017-12-07', '2017-12-02', '2017-11-08',
        '2017-11-01', '2017-10-02', '2017-09-02', '2017-08-02',
        '2017-07-02', '2017-06-02', '2017-05-02', '2017-04-02',
        '2017-03-02', '2017-02-02', '2017-01-01', '2016-12-03',
        '2016-11-02', '2016-10-01', '2016-09-02', '2016-08-02',
        '2016-07-02', '2016-06-02', '2016-05-02', '2016-04-03',
        '2016-02-02', '2015-12-02', '2015-11-01', '2015-09-02',
        '2015-05-04'], dtype=object))

In [0]:
listings = readin_gz(idx[0],'listings')    
listings['file_scraped_dt'] = idx[0]
listings.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,file_scraped_dt
0,958,https://www.airbnb.com/rooms/958,20190503153007,2019-05-03,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,...,f,moderate,f,f,1,1,0,0,1.55,2019-05-03


In [0]:
for i, dt in enumerate(idx[1:]):
    df = readin_gz(idx[i+1],'listings')
   
    df['file_scraped_dt'] = dt
    print(dt,df['last_scraped'].value_counts())
    listings = pd.concat([listings, df], ignore_index = True,axis = 0,sort=False)
        

2019-04-03 2019-04-03    7276
2019-04-05       1
Name: last_scraped, dtype: int64
2019-03-06 2019-03-06    7151
Name: last_scraped, dtype: int64
2019-02-01 2019-02-01    7198
Name: last_scraped, dtype: int64
2019-01-09 2019-01-09    7251
Name: last_scraped, dtype: int64
2018-12-06 2018-12-06    7072
Name: last_scraped, dtype: int64
2018-11-03 2018-11-03    6981
Name: last_scraped, dtype: int64
2018-10-03 2018-10-03    6807
Name: last_scraped, dtype: int64
2018-09-08 2018-09-08    6529
Name: last_scraped, dtype: int64
2018-08-06 2018-08-06    6633
Name: last_scraped, dtype: int64
2018-07-05 2018-07-05    4693
Name: last_scraped, dtype: int64
2018-05-09 2018-05-09    4774
Name: last_scraped, dtype: int64
2018-04-06 2018-04-07    3593
2018-04-06    1251
Name: last_scraped, dtype: int64
2018-03-04 2018-03-04    4802
2018-03-06       2
Name: last_scraped, dtype: int64
2018-02-02 2018-02-02    4727
Name: last_scraped, dtype: int64
2018-01-17 2018-01-17    5224
Name: last_scraped, dtype: int6

In [0]:
# listings_t = listings.set_index(pd.DatetimeIndex(listings['file_scraped_dt']))
# listings_t.head(3)

In [0]:
# # listings['file_scraped_dt'].value_counts()

# m_ls = listings.resample('D').size().reset_index(name='Counts') 
# m_ls = m_ls[m_ls['Counts']>0]
# m_ls

# It appears that the data was scraped twice in each month in Nov 2017, Dec 2017, Jan 2018 prior to the deadline of the new regulations.

In [0]:
ls_cols = listings.columns

listings['is_duplicated'] = listings.duplicated(ls_cols)
listings['is_duplicated'].sum()

listings_u = listings[listings['is_duplicated'] == False].drop(['is_duplicated'],axis=1)
listings_u.shape

0

(334790, 107)

In [0]:
# listings_u = listings_u.set_index(pd.DatetimeIndex(listings_u['last_scraped']))

# d_ls = listings_u.resample('D').size().reset_index(name='Counts') 
# d_ls[d_ls['Counts']>0]

In [0]:
pickle.dump(listings_u, open('all_SF_listings.p','wb'))

In [0]:
listings_u.shape

(334790, 107)

In [0]:
links_sf_reviews = gather_links('san-francisco','reviews.csv.gz')
links_sf_reviews['dt'] = [x.split('/')[6] for x in links_sf_reviews['link']]
links_sf_reviews.head()

idx = links_sf_reviews['dt'].unique()
len(idx),idx

,link,dt
7531,http://data.insideairbnb.com/united-states/ca/...,2019-05-03
7538,http://data.insideairbnb.com/united-states/ca/...,2019-04-03
7545,http://data.insideairbnb.com/united-states/ca/...,2019-03-06
7552,http://data.insideairbnb.com/united-states/ca/...,2019-02-01
7559,http://data.insideairbnb.com/united-states/ca/...,2019-01-09


(45, array(['2019-05-03', '2019-04-03', '2019-03-06', '2019-02-01',
        '2019-01-09', '2018-12-06', '2018-11-03', '2018-10-03',
        '2018-09-08', '2018-08-06', '2018-07-05', '2018-05-09',
        '2018-04-06', '2018-03-04', '2018-02-02', '2018-01-17',
        '2018-01-10', '2017-12-07', '2017-12-02', '2017-11-08',
        '2017-11-01', '2017-10-02', '2017-09-02', '2017-08-02',
        '2017-07-02', '2017-06-02', '2017-05-02', '2017-04-02',
        '2017-03-02', '2017-02-02', '2017-01-01', '2016-12-03',
        '2016-11-02', '2016-10-01', '2016-09-02', '2016-08-02',
        '2016-07-02', '2016-06-02', '2016-05-02', '2016-04-03',
        '2016-02-02', '2015-12-02', '2015-11-01', '2015-09-02',
        '2015-05-04'], dtype=object))

In [0]:
reviews = readin_gz(idx[0],'reviews')       

for i, dt in enumerate(idx[1:]):
    df = readin_gz(idx[i+1],'reviews')
    print(dt)
    reviews = pd.concat([reviews, df], ignore_index = True,axis = 0,sort=False)
        

2019-04-03
2019-03-06
2019-02-01
2019-01-09
2018-12-06
2018-11-03
2018-10-03
2018-09-08
2018-08-06
2018-07-05
2018-05-09
2018-04-06
2018-03-04
2018-02-02
2018-01-17
2018-01-10
2017-12-07
2017-12-02
2017-11-08
2017-11-01
2017-10-02
2017-09-02
2017-08-02
2017-07-02
2017-06-02
2017-05-02
2017-04-02
2017-03-02
2017-02-02
2017-01-01
2016-12-03
2016-11-02
2016-10-01
2016-09-02
2016-08-02
2016-07-02
2016-06-02
2016-05-02
2016-04-03
2016-02-02
2015-12-02
2015-11-01
2015-09-02
2015-05-04


In [0]:
reviews.shape

(10302048, 6)

In [0]:
rv_cols = reviews.columns

In [0]:
reviews['is_duplicated'] = reviews.duplicated(rv_cols)
reviews['is_duplicated'].sum()

reviews_u = reviews[reviews['is_duplicated'] == False].drop(['is_duplicated'],axis=1)
reviews_u.shape

9651267

(650781, 6)

In [0]:
reviews_u.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...


In [0]:
pickle.dump(reviews_u, open('all_SF_reviews.p','wb'))

In [7]:
links_sf_calendar = gather_links('san-francisco','calendar.csv.gz')
links_sf_calendar['dt'] = [x.split('/')[6] for x in links_sf_calendar['link']]
links_sf_calendar.head()

idx = sorted(links_sf_calendar['dt'].unique())
for k, dt in enumerate(idx):
  print(k, dt)

0 2015-05-04
1 2015-09-02
2 2015-11-01
3 2015-12-02
4 2016-02-02
5 2016-04-03
6 2016-05-02
7 2016-06-02
8 2016-07-02
9 2016-08-02
10 2016-09-02
11 2016-10-01
12 2016-11-02
13 2016-12-03
14 2017-01-01
15 2017-02-02
16 2017-03-02
17 2017-04-02
18 2017-05-02
19 2017-06-02
20 2017-07-02
21 2017-08-02
22 2017-09-02
23 2017-10-02
24 2017-11-01
25 2017-11-08
26 2017-12-02
27 2017-12-07
28 2018-01-10
29 2018-01-17
30 2018-02-02
31 2018-03-04
32 2018-04-06
33 2018-05-09
34 2018-07-05
35 2018-08-06
36 2018-09-08
37 2018-10-03
38 2018-11-03
39 2018-12-06
40 2019-01-09
41 2019-02-01
42 2019-03-06
43 2019-04-03
44 2019-05-03


In [8]:
calendar = readin_gz(idx[0],'calendar')     
calendar['file_scraped_dt'] = idx[0]


for i, dt in enumerate(idx[1:24]):
    df = readin_gz(idx[i+1],'calendar')
    df['file_scraped_dt'] = dt
    print(dt, df['file_scraped_dt'].value_counts())
    calendar = pd.concat([calendar, df], ignore_index = True,axis = 0,sort=False)
    
pre_Nov2017_cal = calendar.copy()
pre_Nov2017_cal['file_scraped_dt'].value_counts()

2015-09-02 2015-09-02    2321765
Name: file_scraped_dt, dtype: int64
2015-11-01 2015-11-01    2565585
Name: file_scraped_dt, dtype: int64
2015-12-02 2015-12-02    2615225
Name: file_scraped_dt, dtype: int64
2016-02-02 2016-02-02    3029135
Name: file_scraped_dt, dtype: int64
2016-04-03 2016-04-03    2924380
Name: file_scraped_dt, dtype: int64
2016-05-02 2016-05-02    2951390
Name: file_scraped_dt, dtype: int64
2016-06-02 2016-06-02    3080600
Name: file_scraped_dt, dtype: int64


KeyboardInterrupt: ignored

In [0]:
calendar.shape[0]/1000000 

(122200175, 8)

In [0]:
cal_cols = calendar.columns
cal_cols

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

In [0]:
calendar['is_duplicated'] = calendar.duplicated(cal_cols)
calendar['is_duplicated'].sum()

calendar_u = calendar[calendar['is_duplicated'] == False].drop(['is_duplicated'],axis=1)
calendar_u.shape

81501098

(40699077, 7)

In [0]:
calendar_u.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,34158472,2019-05-03,f,$399.00,$399.00,2.0,1125.0
1,34158472,2019-05-04,f,$399.00,$399.00,2.0,1125.0
2,34158472,2019-05-05,t,$399.00,$399.00,2.0,1125.0
3,34158472,2019-05-06,t,$399.00,$399.00,2.0,1125.0
4,34158472,2019-05-07,t,$399.00,$399.00,2.0,1125.0


In [0]:
pickle.dump(calendar_u, open('all_SF_calendar.p','wb'))

In [0]:
# listings['has_availability'].value_counts()

In [0]:
#  pd.crosstab(listings_u['property_type'].str.upper(), listings_u['room_type'].str.upper(), margins=True)

In [0]:
# len(listings_u['host_id'].unique())

In [0]:
# len(listings_u['id'].unique())

In [0]:
# listings_u['neighbourhood'].str.upper().value_counts().reset_index()

In [0]:
# listings_u['zipcode'] = listings_u['zipcode'].astype(str).str.zfill(5)[:-2]

In [0]:
# listings_u = listings_u.set_index(pd.DatetimeIndex(listings_u['last_scraped']))

In [0]:
# m_ls = listings_u.resample('M').size().reset_index(name='Counts') 
# m_ls = m_ls[m_ls['Counts']>0]
# m_ls

In [0]:
# pd.crosstab(listings_u['last_scraped'], listings_u['has_availability'],margins=True)